# Quantum Cryptography : BB84 protocol 

**[BB84](https://en.wikipedia.org/wiki/BB84)** protocol was set up in 1984 by one of our famous IBM'ers Charles Bennett with his colleague Gilles Brassard. 
It has been experimented a few years later in the first demonstration of quantum key distribution  : [quantum key distribution](https://en.wikipedia.org/wiki/Quantum_key_distribution) by Charles Bennett and John Smolin in IBM [C. H. Bennett, F. Bessette, G. Brassard, L. Salvail, and J. Smolin, J. of Cryptography **5**, 3 (1992) ]. Charles and John are still part of the quantum IBM team. 

<img src="charlie_john_qkd.jpg" width="400"/>
<center>Charles Bennett and John Smolin at T.J. Watson IBM research center.</center>

BB84 protocol allows to communicate a cryptographie key from one point to another and knowing that it has not been compromised. 


## BB84 Protocol 

The steps are the following : 

1. Alice chooses two random bit strings of length  $n$ : $k$ and $b$. The list $k$ has the key value. The list $b$ represents the bases choice for Alice to encode the bits of $k$. When  $b_i=0$ (meaning if the  $i^{th}$ base bit is zero ), she encodes the $i^{th}$ qubit in the standard base $\{|0\rangle, |1\rangle \}$, and if $b_i=1$, she uses the base  $\{|+\rangle, |-\rangle \}$, where $|+\rangle:=\frac{1}{\sqrt{2}}(|0\rangle +|1\rangle)$, $|-\rangle:=\frac{1}{\sqrt{2}}(|0\rangle -|1\rangle)$. 

This can be showned as follows : 


<img src="encoding_Alice.png" alt="drawing" width="300"/>

2. After endoding $n$ bits, Alice sends them to Bob. He chooses a random bit string $\tilde{b}$ of length $n$ for the measurement bases he will be using. Bob records his measurement results  $\tilde{k_i}$ along with the bases used $\tilde{b_i}$, in an array.

3. Then Alice and Bob compare their bases $b_i$ and $\tilde{b}_i$. Whenever  $b_i \neq \tilde{b}_i$, Bob did not measure in the same base Alice did encode. The probability of having measured the correct value for $k_i$ is $\frac{1}{2}$, in this case this position is discarded. Howeever, if $b_i = \tilde{b}_i$, then the qubit was preepared and measured in the same basis (and if noboby did spy on the communication) Bob did measure a correct value : $\tilde{k}_i = k_i$. These values make the key 

## Example : 

Let's assume Alice random key value is :  $k=`0111001`$ and her bases choice : $b=`1101000`$, and let's assume Bob's bases are : $\tilde{b}=`1001101`$. Look at the values below and note that when bases eare not hte same Bob has one chance out of two to get a correct measurement. 
<img src="example_bb84.png" alt="drawing" width="300"/>

In this case, the key is '0110', and to know if it has been seen by Eve, Alice and Bob will have to sacrifice some of theses bits. If a measurement was made during the distribution, le qubit state would have change with probability $\frac{1}{4}$. By verifying $m$ bits values, the probability of not detecting Eve decays as $\left(\frac{3}{4}\right)^m$. So if the verify enough bits, they will gain confidence that Eve did not spy. 

### Message encryption

One the key is known and secretely has been secretly distributed, Alice and Bob can use it very easily  :  [Masque Jetable](https://fr.wikipedia.org/wiki/Masque_jetable) : Alice just adds bitwise modulo 2 the two lists.  (in other words she performs bitwise XOR). With the above example, key is $\text{key}=`0110`$. Secret message is $m=`1100`$, so encrypted message is : $c=m\oplus \text{key (mod 2)} = `1010`$. Bob can decipher in doing the same thing : add the key to the message : $m=c\oplus \text{key (mod 2)}$.

## workshop : 

In this notebook you will be in Bob's role, and with Alice you will generate a key unsing BB84 protocol, then you will beable to decodee Alice's message.

---

There will be 4 steps : 

1. Alice prepares her key and bases. The function (provided) `alice_prepare_qubit` is used to send to you the qubits states. 

2. Bob (you) measure the recevied qubit states in your basis list, you will have to code the function  `build_bob_measure_circuit`. This function receives an index to point in the qubits and in Bob's basis list as well as in the quantum circuit list (which has prepared Alice's qubit state). The function returns a quantum circuit which can be appended to Alice's circuit used to prepare her qubit. Bob circuit simply performs a measure about Bob's base, for each index position. 

3. Then you will have to build the code for making the key Alice and Bob will agree on (by discarding the values that have been emitted and received on non identical basis).

4. Finally, we will be able to write the function to decipher Alice's massage with the key. You generate a bit string, which is a morse code text, you will need to make it readable. 

---

This workhop is derived from the IBM Quantum Computing Challenge, on may 4th, 2020. 

### Setup

In [1]:
# import libraries
import BB84HO
from random import randint
from qiskit import QuantumCircuit, execute, Aer
backend = Aer.get_backend('qasm_simulator')

bases_length = 1600

### Alice prepares her key and her bases

In [2]:
# clef aléatoire
alice_key = ""
for i in range(bases_length): 
    alice_key += str(randint(0,1))

# bases aléatoires
alice_bases = ""
for i in range(bases_length):
    if randint(0,1):
        alice_bases += "z"
    else:
        alice_bases += "x"

### Bob prepares his bases : 

In [3]:
bob_bases = ""
for i in range(bases_length):
    if randint(0,1):
        bob_bases += "z"
    else:
        bob_bases += "x"
        
print('Bob\'s bases :', bob_bases)

Bob's bases : zzzzxxxxxzzzxzxxxxxxxzxxzzzzzzzzzzxzzzxzxzxzxxxzxxzzxzzxzzxxxzxzzxzzxxzxzzxzzzzzxzzxzxxzxzxxzxzxzxxzzxzzzzzxxxzxxxzxzzxzxxxzzzxxxxzxzzxxzzxzxzxzzzxzxzxxzzxzzxzxzxzxzxxxzzzzxzzzxxxxzzxzzzzzxzxxxxxxzzxxzxzzzxxzzxxzzzxxxxzzxxzxxxzxzxzzxzzxxzzzzxzzxxxxxxxzxzzzzzxxzzxzzxzxzzzxxzzzzzxxzzzzxxzxxzzzzxxxzxzzzzzxzxzxxxxzzxzzzzzxzxzxzxxzxxxxzzxxxxxzxzzzzzzxzzzxxxzzxxxzxzzxzxzzxxxxxzxxzxxxzzxxzxzxxxzzxzxzxzxxzxxxxxzxxxzxxzxzzxzxxzzzxxxxzzxzxxxzzzzxzxzzzxxzxxxzxxxxzzzxzzzxxxzxzzxxxxxzzzzzxxxxzzxzzzzxxxzzzxzxxzxxxzxzzxxzzzxxxxxzxzzzxzxzzzzzzxzxzxzzxzzzzxzxzxzzxxzzxzxzzxxzxxxzzxzxzzxzzxxxxzzxxxxzzxzxzzzzzxzxzzxzxxxxxzzzzzxzzzzzxzxzzzzzzzxxxxzzxzxzxxzzzxzxxzxxzxxzxzxzzxxzzxxxzxxzzzxzxxxzxxzxzzxxzzxzzzxxxzxzxzxzzxzxxxxzxzxxxzxxzxzzzzxzxzzzzxzzzzxzzzzzzxzzzzzzzzxxzxzzzxzxxxxxxzxzzxxzzzzzxzxzzzxzzxzxzzzxzxxxzzzxxzzzzzxxxzzxxzzzxxzzxzzxzxzxzxzxxzxzxzxzxzxxzxxzxxxzzxxzzxzxzxzxxxzxzxxzxxzxxzxzzxxxxxzzzzzzxzzzzxzxxxxzxzxxzzxxxxxxxxzxzzxzzzxzxxxxxxzzxzzxzxzxzzzzxzzxxzxxzxxzzxxzxzzxxzzzxxxxzxzzxx

### Now Bob performs the measurements (with his bases) on states prepared by Alice (with her key and bases): 


In [4]:
bob_results = ""

def bb84():
    print('Bob\'s bases:', bob_bases)

    # now Alice prepares her qubits one at a time, using her basis list, 
    # Bob measures, using his basis list.
    bob_results = ""
    
    # for each bit
    for index in range(bases_length):  # on va faire ceci 1600 fois une fois pour chaque état
        # Alice create her qubit state
        thisqubit_circuit = BB84HO.prepare_alice_qubit(index, alice_bases, alice_key)
        
        # Bob prepares for measure (function to be coded below) 
        bob_circ = build_bob_circuit(index, bob_bases, thisqubit_circuit)
        
        # now we execute and measure on the simulator 
        bob_job = execute(bob_circ, backend=backend, shots=1)
        bob_reading = bob_job.result()
        
        # for each result we build the list
        bob_results += list(bob_reading.get_counts(bob_circ))[0]
    return bob_results

# this is the function to be writen : 
# you need to "continue" a quantum cicruit sent by Alice (qc) using one qubit so that you
# make the measurement using Bob's basis at index = index.
# the function returns the quantum circuit. 

def build_bob_circuit(index, bob_bases, qc):    
    # WRITE YOR CODE HERE --------
  


    # END ------------------------ 
    
bob_bits = bb84()        
print('Bob\'s bits: ', bob_bits)


Bob's bases: zzzzxxxxxzzzxzxxxxxxxzxxzzzzzzzzzzxzzzxzxzxzxxxzxxzzxzzxzzxxxzxzzxzzxxzxzzxzzzzzxzzxzxxzxzxxzxzxzxxzzxzzzzzxxxzxxxzxzzxzxxxzzzxxxxzxzzxxzzxzxzxzzzxzxzxxzzxzzxzxzxzxzxxxzzzzxzzzxxxxzzxzzzzzxzxxxxxxzzxxzxzzzxxzzxxzzzxxxxzzxxzxxxzxzxzzxzzxxzzzzxzzxxxxxxxzxzzzzzxxzzxzzxzxzzzxxzzzzzxxzzzzxxzxxzzzzxxxzxzzzzzxzxzxxxxzzxzzzzzxzxzxzxxzxxxxzzxxxxxzxzzzzzzxzzzxxxzzxxxzxzzxzxzzxxxxxzxxzxxxzzxxzxzxxxzzxzxzxzxxzxxxxxzxxxzxxzxzzxzxxzzzxxxxzzxzxxxzzzzxzxzzzxxzxxxzxxxxzzzxzzzxxxzxzzxxxxxzzzzzxxxxzzxzzzzxxxzzzxzxxzxxxzxzzxxzzzxxxxxzxzzzxzxzzzzzzxzxzxzzxzzzzxzxzxzzxxzzxzxzzxxzxxxzzxzxzzxzzxxxxzzxxxxzzxzxzzzzzxzxzzxzxxxxxzzzzzxzzzzzxzxzzzzzzzxxxxzzxzxzxxzzzxzxxzxxzxxzxzxzzxxzzxxxzxxzzzxzxxxzxxzxzzxxzzxzzzxxxzxzxzxzzxzxxxxzxzxxxzxxzxzzzzxzxzzzzxzzzzxzzzzzzxzzzzzzzzxxzxzzzxzxxxxxxzxzzxxzzzzzxzxzzzxzzxzxzzzxzxxxzzzxxzzzzzxxxzzxxzzzxxzzxzzxzxzxzxzxxzxzxzxzxzxxzxxzxxxzzxxzzxzxzxzxxxzxzxxzxxzxxzxzzxxxxxzzzzzzxzzzzxzxxxxzxzxxzzxxxxxxxxzxzzxzzzxzxxxxxxzzxzzxzxzxzzzzxzzxxzxxzxxzzxxzxzzxxzzzxxxxzxzzxxx

### After Bob made his measurement, Alice and Bob share their bases: 


In [ ]:
print("alice_bases = ", alice_bases, "\n")
print("bob_bases   = ", bob_bases)

### Now Alice and Bob can construct their key :
They will keep only the bits corresponding to index where the bases were identical. 


In [6]:
# conserver les bits de la clef d'Alice (ou de ceux qu'a lu Bob) si et seulement si la base choisie
# par Alice et la base choisie par Bob étaient les mêmes 
bob_key = ""
ali_key = ""
# START : write your code here 
for i in range(bases_length): 
    if bob_bases[i] == alice_bases[i]: 
        bob_key += bob_bits[i]
        ali_key += alice_key[i]
    

# END ------------------------ 


print(len(bob_key))
print("bob_key= ", bob_key)
print("ali_key= ", ali_key)


794
bob_key=  11011110001001010011101111010011111110101010111010000111010100101000000100100100010001000011010101111101001001001011000001001001111000100000100011011100001011111010111110101001011100101010101101111000001101000011011110011011010011000000111101101100111100011010011110001010110011101011010101100010000001001111011011000100001010010011000111101101100001000010101111110110000110001101001010010001100000010010110100000101101110011110001001001110000100011010011010100110110111010011101000001110010011100010000000011111111010111110110010001001100010111101110111110111110101111011011101001000011001110011111111000111000110100101100000001011011000011000110111101110111001011111111111000001010010011011010011011101101000110101110001000110000010000110101100100011011001110001010100101001101100111010011101
ali_key=  1110111010110100010100111101011111011010111011101000010111110010010110010110011001011101000000010111101101111101111110110100100111100110000011001110110001110111111011110011100101111

### Now Alice and Bob should have the same Key.... 

### let them verify using 50 first bits .... if Eve was listening

Remember the probability of Eve listening and not being detected is $(\frac{3}{4})^{50} \approx 6.10^{-7}$


In [ ]:
print(ali_key[0:50])
print(bob_key[0:50])
err_count = 0
for i in range(50):
    if ali_key[i] !=  bob_key[i]:
        err_count += 1 
print(err_count)

if err_count > 3: 
    print("Attention attention attention !!! ")
    print("Eve a écouté, il faut tout recommencer") 
    print("il faut redémarrer l'éxecution -> restart kernel")
else:
    print("Bonne nouvelle, Eve n'a pas écouté, ")
    print("mais attention, les 50 premiers bits")
    print("de la clef ont été communiqués en clair,")
    print("il ne faut pas s'en servir")

In [ ]:
# dans cette celulle modifiez bob_key et ali_key pour enlever les 50 premiers bits qui ont été compromis


#

### Now Alice can use the key to encode her message : 

execute the following cell

In [ ]:
mess = BB84HO.code(BB84HO.alice_message_clair, ali_key)

print(mess)
print(len(mess))

### Function to decode the message using the key: 

In [ ]:
# this is Alice message : print("message :", mess)
print(len(mess))

# here we decode Alice message, using our key
def decode(message,clef):
       
    # ensuite composez le message "en clair" en faisant le XOR bit à bit entre le message et la clef
    # il s'agit de l'addition bit à bit : (a + b) % 2 , si a vaut 0 ou 1 et b vaut 0 ou 1. 
    # retournez la variable "clair" contenant le message décodé
    
    # START : write your code here 
   




    
    # END ------------------------ 

    
clair = decode(mess,bob_key)
clair = clair.rstrip("0")
print("clair :" ,clair)

Message is in morse code :

- point : '1' ("ti")
- dash : '11' ("ta")
- separator : '0'
- letter separator :  '00'
- word separator: '000'.

For example :
<img src="qiskit_morse.png" alt="drawing" width="500"/>

### Try and decode Alice message  !

One usefull Python method can be "split()" : `string.split(separator, maxsplit)`

You may "split" the string into a word list `texte.split("000")`, then split words into letters with `mot.split("00")`, finally split letters into signs ("ti" and "ta"), then pick up each letter from the provided dictionnary to display the message. 


In [ ]:
M = { '.-':'a', '-...':'b', '-.-.':'c', '-..':'d', '.':'e', '..-.':'f', '--.':'g', 
     '....':'h', '..':'i', '.---':'j', '-.-':'k', '.-..':'l', '--':'m', '-.':'n',
     '---':'o', '.--.':'p', '--.-':'q', '.-.':'r', '...':'s', '-':'t', '..-':'u',
     '...-':'v', '.--':'w', '-..-':'x', '-.--':'y', '--..':'z', '.----':'1', 
     '..---':'2', '...--':'3', '....-':'4', '.....':'5', '-....':'6', '--...':'7',
     '---..':'8', '----.':'9', '-----':'0', '--..--':',', '.-.-.-':'.', '..--..':'?',
     '-..-.':'/', '-....-':'-', '--..--':',', '---...':':', '-.--.':'(', '-.--.-':')',
     '..-..':'é','.--.-':'à', '-.-.--':'!'} 

# ecrivez une fonction qui rend lisible le code morse de "clair"



# END ------------------------ 